# Demo 1: An Introduction to PyCUDA
## EECS E4750: Heterogenous Computing for Signal and Data Computing
Fall 2020

## Preface

Welcome to E4750! In this Jupyter Notebook, you will explore what may very well be your first encounter with CUDA programming, through the popular Python wrapper - **PyCUDA**.

While I am showcasing this demo through the use of a Jupyter Notebook, please keep in mind that assignments and even future recitations will eschew these - we will work exclusively with executable python scripts, not jupyter notebooks. Nevertheless, this and the PyOpenCL demo will be available on the course repo if you need to revisit either.

## The CUDA Device and APIs

### CUDA Initialization
The first step is obviously to import the PyCUDA wrapper. The `pycuda.driver.init` method initializes CUDA. In all cases, this must always be the first step. Alternatively, you could try `pycuda.driver.autoinit`, which performs initialization as well as context creation.

### CUDA Runtime and Driver APIs

There are two APIs associated with what one collectively refers to as CUDA. These are the CUDA runtime and CUDA driver APIs. As the [CUDA Toolkit Documentation](https://docs.nvidia.com/cuda/cuda-driver-api/driver-vs-runtime-api.html#driver-vs-runtime-api) notes, the driver and runtime are quite similar and can infact be used interchangeably, although there are some noteworthy differences that separate the two.

The runtime API provides some ease with implicit initialization, context management and module management. The driver API on the other hand, provides finer control over contexts and module loading. With the runtime, all kernels are automatically loaded during initialization and remain loaded as long as the program using those kernels is running. With the driver API, it is possible to retain modules on an as-needed-basis, and also to dynamically reload modules without re-initialization.

The CUDA runtime API is a wrapper of the CUDA driver API.

### Kernels

CUDA C/C++ extends C/C++ by allowing one to define special functions, called *kernels*. When called, these are executed *N* times in parallel by *N* different CUDA threads, unlike normal host code functions, which would only be executed once upon being called.

### Contexts

Contexts can be thought of as the bridge between your host (loosely, the CPU) and the CUDA Device (more specifically, the Runtime API). All communication with the GPU takes place within and through the context. The context holds all management data to control and use the CUDA Device.; For example, it keeps track of allocated memory (because you do not want to run out of it or allocate more than physically available), the loaded modules that contain device code (i.e. kernels), and the mapping between host (CPU) and GPU memory.

Why is this necessary? GPU device resources are actually scarce (indeed, the raw power of a CPU is usually greater than a GPU) and one can never have enough. Actual abundance in numbers of CUDA cores is not relevant if they are inaccessible. Therefore, you can think of the Context as a life-cycle manager - it manages the existence of objects in typical CUDA programs, such as:
    * memory buffers
    * modules
    * functions
    * CUDA streams
    * events
    * and more.

The driver API lets you manage contexts, while these are not exposed to the runtime API. Instead, the runtime API dynamically decides which context to use for a process. There can only be one active context on the device at a time, and the objects within it are destroyed with the context when ended, so they are usable only within the context in which they were created.

All this to say, a single context is created and used for when an on-device program is run. Since these programs will constitute one or more kernels, the key takeaway is that those specific kernels can be used only in the context they were loaded through. Once the context is gone, the kernels/program would need to be built and sourced again for the next context.


For simplicity, let's first import pycuda and interact with the API to fetch some information about the CUDA device and the driver itself.

In [ ]:
import pycuda
import pycuda.driver as cuda

cuda.init()
#cuda.autoinit()

Let's check the number of CUDA devices visible to the driver:

In [ ]:
print("Number of CUDA devices available: ", cuda.Device.count())

Number of CUDA devices available:  1


One! This is unsurprising, since I am running this on a laptop with just one Nvidia GPU. If you ran this on a multi-GPU machine (lucky you!), you would see a different count.

In [ ]:
my_device = cuda.Device(0)

# cc: compute capability
cc = float('%d.%d' % my_device.compute_capability())
print('Device Name: {}'.format(my_device.name()))
print('Compute Capability: {}'.format(cc))
print('Total Device Memory: {} megabytes'.format(my_device.total_memory()//1024**2))


Device Name: GeForce RTX 2070
Compute Capability: 7.5
Total Device Memory: 7982 megabytes


In [ ]:
# In CUDA/C, deviceQuery will display a bit more
# detail. To see these using PyCUDA, we will set up
# a python dictionary to index the values
# device.get_attributes will provide.

my_dev_attributes_tuples = my_device.get_attributes().items()
mydev_attributes = {}

for key, value in my_dev_attributes_tuples:
    mydev_attributes[str(key)] = value

for k in mydev_attributes.keys():
    print('\t {}: {}'.format(k, mydev_attributes[k]))



ASYNC_ENGINE_COUNT: 3
	 CAN_MAP_HOST_MEMORY: 1
	 CLOCK_RATE: 1440000
	 COMPUTE_CAPABILITY_MAJOR: 7
	 COMPUTE_CAPABILITY_MINOR: 5
	 COMPUTE_MODE: DEFAULT
	 CONCURRENT_KERNELS: 1
	 ECC_ENABLED: 0
	 GLOBAL_L1_CACHE_SUPPORTED: 1
	 GLOBAL_MEMORY_BUS_WIDTH: 256
	 GPU_OVERLAP: 1
	 INTEGRATED: 0
	 KERNEL_EXEC_TIMEOUT: 1
	 L2_CACHE_SIZE: 4194304
	 LOCAL_L1_CACHE_SUPPORTED: 1
	 MANAGED_MEMORY: 1
	 MAXIMUM_SURFACE1D_LAYERED_LAYERS: 2048
	 MAXIMUM_SURFACE1D_LAYERED_WIDTH: 32768
	 MAXIMUM_SURFACE1D_WIDTH: 32768
	 MAXIMUM_SURFACE2D_HEIGHT: 65536
	 MAXIMUM_SURFACE2D_LAYERED_HEIGHT: 32768
	 MAXIMUM_SURFACE2D_LAYERED_LAYERS: 2048
	 MAXIMUM_SURFACE2D_LAYERED_WIDTH: 32768
	 MAXIMUM_SURFACE2D_WIDTH: 131072
	 MAXIMUM_SURFACE3D_DEPTH: 16384
	 MAXIMUM_SURFACE3D_HEIGHT: 16384
	 MAXIMUM_SURFACE3D_WIDTH: 16384
	 MAXIMUM_SURFACECUBEMAP_LAYERED_LAYERS: 2046
	 MAXIMUM_SURFACECUBEMAP_LAYERED_WIDTH: 32768
	 MAXIMUM_SURFACECUBEMAP_WIDTH: 32768
	 MAXIMUM_TEXTURE1D_LAYERED_LAYERS: 2048
	 MAXIMUM_TEXTURE1D_LAYERED_WIDTH

## Demo PyCUDA program

Let us explore a simple PyCUDA program that doubles the elements of an input numpy array.


In [ ]:
import pycuda.autoinit
from pycuda.compiler import SourceModule
import numpy as np

In [ ]:
# Create a 4x4 2D numpy array

a = np.random.randn(4,4)
a = a.astype(np.float32)

### Memory Allocation and Host-to-Device Transfer

This array `a` is currently on the host memory, so no GPU operation can be performed on it. (To be pedantic, PyCUDA will still let you do this but if your host program were written in C/C++, your kernel won't launch - since there is no variable in the device memory to operate on. You will investigate the ramifications of this requirement in Assignment-1.)

So, the first step is to allocate *exactly* amount of memory in device memory that the FP32 variable `a` occupies in host memory. Above, we forced the numpy array `a` to be of the data type `numpy.float32`. This is because GPUs, unlike CPUs, do not support 64 bit floating point precision - only 32. So, for variables created on the host, this requirement must be enforced to avoid trouble.

In [ ]:
a_gpu = cuda.mem_alloc(a.size * a.dtype.itemsize)

Just allocating memory isn't enough - you need to actually copy the array to device memory from its host location to the allocated memory on the device.

In [ ]:
cuda.memcpy_htod(a_gpu, a)

### Kernel Code

Next, we need a CUDA kernel that defines exactly what the GPU is to do with the array that we are placing in it's memory. With CUDA C/C++, this is quite simple. But since we're using PyCUDA, the process is a little different. While the majority of peripheral code you will encounter in assignments will by pythonic, the kernel code itself **must be written in C syntax**. Hence, we let the driver create a module for our kernel in CUDA source for the current context.

In [ ]:
mod = SourceModule("""
    __global__ void doublify(float *a)
    {
      int idx = threadIdx.x;
      a[idx] *= 2;
    }
    """)

Here, `__global__` indicates the function is a CUDA kernel function – called by the host and executed on the
device, while `void` means that the kernel doesn't return anything.

* `doublify`: Kernel function name
* `float *a`: The kernel function arguments. `*a` because we're working with a pointer to the device memory.

CUDA kernel execution is done in chunks, called thread-blocks/blocks. Each device has a specific maximum blocksize i.e. a maximum number of threads that can be run concurrentlyu for any block. You can additionally define a grid of multiple blocks. We will ignore this for now.


### Thread Indexing

Let's look closely at the second line of the kernel function.

* `int idx = threadIdx.x + threadIdx.y*4;`

This defines a unique thread id among all threads in a grid. What do those individual words mean?

* Threadblocks are 2D - so they will have two dimensions along which to index.
* `threadIdx.x`: specifies the x-index of a local thread id within a 2D thread block
* `threadIdx.y`: specifies the y-index of a local thread id within a 2D thread block

For this example we aren't looking at grids; but if we were, there is an additional detail to understand. Consider another example of 1D indexing (for a completely unrelated kind of kernel):
`int i = blockDim.x * blockIdx.x + threadIdx.x;`

* `blockDim`: gives the number of threads within each block (in the x-dimension in 1D case)
* `blockIdx`: specifies which block the thread belongs to (within the grid of blocks)


### Where's the for loop?

So if you're wondering where the for loop is to define which index has to be worked on when, you're asking a pertinent question. Recall that kernels are executed *N* times in parallel by *N* different CUDA threads. The indexing we've defined controls how many threads are handled at once.


### Launching the kernels

The compiled kernel code can be called by using `mod.get_function`.


In [ ]:
func = mod.get_function("doublify")
start = cuda.Event()
end = cuda.Event()

start.record()
func(a_gpu, block=(4,4,1))
end.record() # wait for event to finish

# time event execution in milliseconds
t = start.time_till(end)
# blocksize=(4, 4, 1)

Your block size determines in what order your input array is read by the kernel code. In this case, each block is entirely composed of the input array and nothing else. So, thread indexing was written the way we did:
* `int idx = threadIdx.x + threadIdx.y*4;`

Which means that the kernel executes the block it receives by treating it as a 1D block of 1D length elements. ([Read about row-major indexing](https://en.wikipedia.org/wiki/Row-_and_column-major_order)).

### Fetching the result

Once execution has concluded, the result must be retrieved and stored in host memory to be displayed.

In [ ]:
a_doubled = np.empty_like(a)
cuda.memcpy_dtoh(a_doubled, a_gpu)

In [ ]:
print('Original array: ', a)
print('Doubled array: ', a_doubled)
print('Execution time:', t, ' milliseconds.')

Original array:  [[-0.16816713  0.70489305 -1.1215461  -0.43767273]
 [ 1.4034244  -1.6449759   0.5100117   0.8407693 ]
 [-3.494945   -0.7225162  -0.74454343 -1.0320127 ]
 [ 0.05584498  0.70544595 -1.1988835   2.0706177 ]]
Doubled array:  [[-0.33633426  1.4097861  -2.2430923  -0.87534547]
 [ 1.4034244  -1.6449759   0.5100117   0.8407693 ]
 [-3.494945   -0.7225162  -0.74454343 -1.0320127 ]
 [ 0.05584498  0.70544595 -1.1988835   2.0706177 ]]
Execution time: 0.9976639747619629  milliseconds.
